In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json

In [4]:
image = cv2.imread("1005_20UPGM23211816_AfterBonding.jpg")
image.shape

(5504, 6880, 3)

In [5]:
## Trouver la colonne verte à gauche

def find_column_left(image):
    colonnes=[image[:,i,:] for i in range(image.shape[1])]
    i=0
    bool=False
    while not bool and i<len(colonnes):
        col=colonnes[i]
        for pix in col:
            if (pix[1]>50) and (pix[0]<30) and (pix[2]<30):
                bool=True
        if not bool:
            i+=1
    return i, colonnes[i]

In [6]:
# Même chose à droite

def find_column_right(image):
    colonnes=[image[:,i,:] for i in range(image.shape[1])]
    i=image.shape[1]-1
    bool=False
    while not bool and i>=0:
        col=colonnes[i]
        for pix in col:
            if (pix[1]>50) and (pix[0]<30) and (pix[2]<30):
                bool=True
        if not bool:
            i-=1
    return i, colonnes[i]

In [7]:
# Compte le nombre de fils du module

def count(image, column_left, column_right) :
    state = False
    levels = np.arange(0.6, 1.0, 0.01)
    counters = np.zeros(levels.shape[0])
    for j in range(levels.shape[0]) :
        for i in range(image.shape[0]) :
            if (image[i,column_left-5:column_left+5].sum()/(255*10*3)) > levels[j] :
                if not(state) :
                    counters[j]+=1
                    state = True
            else :
                state = False
    state = False
    for j in range(levels.shape[0]) :
        for i in range(image.shape[0]) :
            if (image[i,column_right-5:column_right+5].sum()/(255*10*3)) > levels[j] :
                if not(state) :
                    counters[j]+=1
                    state = True
            else :
                state = False
    unique, counts = np.unique(counters, return_counts=True)
    arg = np.argmax(counts)
    return (unique[arg])

# counters = count(image, column_left=777, column_right=6080)
# plt.plot(np.arange(0.6, 1.0, 0.01), counters)

print(count(image, column_left=777, column_right=6080))

702.0


In [52]:
image2 = cv2.imread("ModulePictures/20UPGM23211355_PPPV2_46_AfterBonding.jpg")
print(count(image2, column_left=777, column_right=6080))

700.0


In [9]:
image3 = cv2.imread("ModulePictures/P1008_20UPGM23210595_AfterWirebonding.jpg")
print(count(image3, column_left=777, column_right=6080))

700.0


In [10]:
image4 = cv2.imread("ModulePictures/P1014_20UPGM23211689_AfterWirebonding.jpg")
print(count(image4, column_left=840, column_right=6080))

699.0


In [11]:
with open("iref_trim_per_module.json", "r") as f:
    data = json.load(f)

In [42]:
def iref_trim (serialNumber, data) :
    error = True
    for x in data :
        if x['serialNumber'] == serialNumber :
            iref = x
            error = False
    if error :
        return ("serialNumber not found")
    return (iref['IREF_TRIM_1'], iref['IREF_TRIM_2'], iref['IREF_TRIM_3'], iref['IREF_TRIM_4'])

In [43]:
iref_trim('20UPGM23210148', data)

(13, 11, 8, 6)

In [44]:
def extract_serial_number (file_name) :
    names = file_name.split("_")
    for x in names :
        if "20UPGM" in x :
            return (x)
    return (None)

In [ ]:
extract_serial_number("1005_20UPGM23211816_AfterBonding")

'20UPGM23211816'

In [46]:
def expected_wire_number (serialNumber, data) :
    iref = iref_trim(serialNumber, data)
    nb_wire_per_trim = [4, 3, 3, 2, 3, 2, 2, 1, 3, 2, 2, 1, 2, 1, 1, 0]
    return (693 + nb_wire_per_trim[iref[0]] + nb_wire_per_trim[iref[1]] + nb_wire_per_trim[iref[2]] + nb_wire_per_trim[iref[3]])

In [47]:
serial = extract_serial_number("1005_20UPGM23211816_AfterBonding")
iref_trim(serial, data)

(8, 8, 9, 7)

In [48]:
expected_wire_number(serial, data)

702

In [65]:
serial = extract_serial_number("20UPGM23211355_PPPV2_46_AfterBonding")
iref_trim(serial, data)

(11, 9, 6, 10)

In [61]:
def test_wire_number (file_name, data, column_left = 777, column_right = 6080) :
    expected_nb = expected_wire_number(extract_serial_number(file_name), data)
    real_nb = count(cv2.imread("ModulePictures/" + file_name + ".jpg"), column_left, column_right)
    return (expected_nb == real_nb)

In [62]:
test_wire_number("20UPGM23211355_PPPV2_46_AfterBonding", data)

np.True_